In [17]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import logging

# Настройка логирования
logging.basicConfig(level=logging.DEBUG)

# Путь к вашему файлу учетных данных
SERVICE_ACCOUNT_FILE = 'google_sheet_key.json'
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service_docs = build('docs', 'v1', credentials=credentials)

# ID вашего документа Google Docs
DOCUMENT_ID = '1Rtfhw54kpkmuPmA7xpt9CVx5J_70kMiL636MQzl3dck'

# Функция для получения структуры документа
def get_document_structure(document_id):
    try:
        logging.info("Получение текущего содержимого документа")
        document = service_docs.documents().get(documentId=document_id).execute()
        
        # Логирование текущего содержимого документа
        logging.debug(f"Текущее содержимое документа: {document}")

        # Извлечение и вывод структуры документа
        content = document.get('body').get('content')
        for element in content:
            if 'paragraph' in element:
                for paragraph_element in element['paragraph']['elements']:
                    text_run = paragraph_element.get('textRun')
                    if text_run:
                        text_content = text_run.get('content')
                        print(f"Text: {text_content.strip()}")
                    else:
                        print("Other element in paragraph")
            elif 'table' in element:
                print("Table element found")
            elif 'tableOfContents' in element:
                print("Table of contents found")
            else:
                print("Other element type found")

    except Exception as e:
        logging.error(f"Ошибка при получении структуры документа: {e}")

# Получаем и выводим структуру документа
get_document_structure(DOCUMENT_ID)


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:root:Получение текущего содержимого документа
DEBUG:googleapiclient.discovery:URL being requested: GET https://docs.googleapis.com/v1/documents/1Rtfhw54kpkmuPmA7xpt9CVx5J_70kMiL636MQzl3dck?alt=json
DEBUG:google_auth_httplib2:Making request: POST https://oauth2.googleapis.com/token
DEBUG:root:Текущее содержимое документа: {'title': 'nlmk_template_didgest', 'body': {'content': [{'endIndex': 1, 'sectionBreak': {'sectionStyle': {'columnSeparatorStyle': 'NONE', 'contentDirection': 'LEFT_TO_RIGHT', 'sectionType': 'CONTINUOUS'}}}, {'startIndex': 1, 'endIndex': 6, 'paragraph': {'elements': [{'startIndex': 1, 'endIndex': 6, 'textRun': {'content': 'НЛМК\n', 'textStyle': {'bold': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0.42745098, 'green': 0.39215687, 'blue': 0.9098039}}}, 'fontSize': {'magnitude': 20, 'unit': 'PT'}}}}], 'paragraphStyle': {'namedStyleType': 'NORMAL_TEXT', 'direction': 'L

Other element type found
Text: НЛМК
Text: Информационный бюллетень
Text: 4 сентября 20XX г.
Text: 
Other element in paragraph
Text: 
Text: Заголовок новости
Text: 
Text: Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь.
Text: Ссылка на новость
Text: 
Text: Заголовок новости
Text: Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь.
Text: Ссылка на новость
Text: 
Text: Заголовок новости
Text: Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введите свой текст здесь Введит

In [ ]:
rss_df_raw = ggl.extract_data_from_google_sheet('https://docs.google.com/spreadsheets/d/1BA1nioQqc048FFvKXcpP5VqL_73kXNCcSx0m-jhI2MQ/edit#gid=2038294867', 
    0)
rss_df_raw